# Database generator

By Kenneth Burchfiel
Released under the MIT License

This script creates tables with fake student data that can be used in visualization and analysis scripts.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 1000)
from faker import Faker
import numpy as np
import geopandas
import folium
import sqlalchemy

fake = Faker() # https://faker.readthedocs.io/en/master/index.html

In [2]:
rng = np.random.default_rng(seed=1158) 
# https://numpy.org/doc/stable/reference/random/generator.html

In [3]:
# Source of addresses:
# https://nces.ed.gov/programs/edge/geographic/schoollocations

# df_address_list = pd.read_excel('EDGE_GEOCODE_PUBLICSCH_2122.xlsx') # Took
# # around 30 seconds to load
# # Converting to a .csv for faster loading:
# df_address_list.to_csv('school_address_list.csv', index = False)

# df_private_school_address_list = pd.read_excel('EDGE_GEOCODE_PRIVATESCH_1920.xlsx') 
# # Converting to a .csv for faster loading:
# df_private_school_address_list.to_csv('private_school_address_list.csv', index = False)

The NCES has made over 120,000 school addresses and corresponding geographic coordinates publicly available on their website, so I chose to use that data as a source of student addresses.

In [4]:
df_private_school_address_list = pd.read_csv(
    'private_school_address_list.csv', low_memory = False)

df_public_school_address_list = pd.read_csv('school_address_list.csv', low_memory = False)

df_address_list = pd.concat([df_public_school_address_list, df_private_school_address_list])
df_address_list

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,NMCNTY,LOCALE,LAT,LON,CBSA,NMCBSA,CBSATYPE,CSA,NMCSA,NECTA,NMNECTA,CD,SLDL,SLDU,SCHOOLYEAR,PPIN
0,1.000050e+10,100005.0,Albertville Middle School,1.0,600 E Alabama Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.260200,-86.206200,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
1,1.000050e+10,100005.0,Albertville High School,1.0,402 E McCord Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.262200,-86.204900,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
2,1.000050e+10,100005.0,Albertville Intermediate School,1.0,901 W McKinney Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.273300,-86.220100,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
3,1.000050e+10,100005.0,Albertville Elementary School,1.0,145 West End Drive,Albertville,AL,35950,1,1095,Marshall County,32,34.252700,-86.221806,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
4,1.000050e+10,100005.0,Albertville Kindergarten and PreK,1.0,257 Country Club Rd,Albertville,AL,35951,1,1095,Marshall County,32,34.289800,-86.193300,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21567,NaN,NaN,YESHIVA BETH MIKROH,NaN,221 VIOLA RD,MONSEY,NY,10952,36,36087,Rockland County,21,41.135509,-74.078211,35620,"New York-Newark-Jersey City, NY-NJ-PA",1,408,"New York-Newark, NY-NJ-CT-PA",N,N,3617,36098,36038,2019-2020,Y9105109
21568,NaN,NaN,METRO CHRISTIAN ACADEMY,NaN,6363 S TRENTON AVE,TULSA,OK,74136,40,40143,Tulsa County,11,36.069900,-95.965841,46140,"Tulsa, OK",1,538,"Tulsa-Muskogee-Bartlesville, OK",N,N,4001,40071,40035,2019-2020,Y9105326
21569,NaN,NaN,SAYRE SCHOOL,NaN,194 N LIMESTONE,LEXINGTON,KY,40507,21,21067,Fayette County,11,38.047430,-84.493511,30460,"Lexington-Fayette, KY",1,336,"Lexington-Fayette--Richmond--Frankfort, KY",N,N,2106,21077,21013,2019-2020,Z0516931
21570,NaN,NaN,FORT WORTH CHRISTIAN SCHOOL,NaN,6200 HOLIDAY LN,NORTH RICHLAND HILLS,TX,76180,48,48439,Tarrant County,21,32.857358,-97.220487,19100,"Dallas-Fort Worth-Arlington, TX",1,206,"Dallas-Fort Worth, TX-OK",N,N,4812,48091,48009,2019-2020,Z1326859


In [5]:
# Adding leading zeroes back into zip codes:

df_address_list['ZIP'] = df_address_list['ZIP'].astype('str').str.zfill(5)
# Took only 0.4 seconds to load
df_address_list

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,NMCNTY,LOCALE,LAT,LON,CBSA,NMCBSA,CBSATYPE,CSA,NMCSA,NECTA,NMNECTA,CD,SLDL,SLDU,SCHOOLYEAR,PPIN
0,1.000050e+10,100005.0,Albertville Middle School,1.0,600 E Alabama Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.260200,-86.206200,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
1,1.000050e+10,100005.0,Albertville High School,1.0,402 E McCord Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.262200,-86.204900,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
2,1.000050e+10,100005.0,Albertville Intermediate School,1.0,901 W McKinney Ave,Albertville,AL,35950,1,1095,Marshall County,32,34.273300,-86.220100,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
3,1.000050e+10,100005.0,Albertville Elementary School,1.0,145 West End Drive,Albertville,AL,35950,1,1095,Marshall County,32,34.252700,-86.221806,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
4,1.000050e+10,100005.0,Albertville Kindergarten and PreK,1.0,257 Country Club Rd,Albertville,AL,35951,1,1095,Marshall County,32,34.289800,-86.193300,10700,"Albertville, AL",2,N,N,N,N,104,01026,01009,2021-2022,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21567,NaN,NaN,YESHIVA BETH MIKROH,NaN,221 VIOLA RD,MONSEY,NY,10952,36,36087,Rockland County,21,41.135509,-74.078211,35620,"New York-Newark-Jersey City, NY-NJ-PA",1,408,"New York-Newark, NY-NJ-CT-PA",N,N,3617,36098,36038,2019-2020,Y9105109
21568,NaN,NaN,METRO CHRISTIAN ACADEMY,NaN,6363 S TRENTON AVE,TULSA,OK,74136,40,40143,Tulsa County,11,36.069900,-95.965841,46140,"Tulsa, OK",1,538,"Tulsa-Muskogee-Bartlesville, OK",N,N,4001,40071,40035,2019-2020,Y9105326
21569,NaN,NaN,SAYRE SCHOOL,NaN,194 N LIMESTONE,LEXINGTON,KY,40507,21,21067,Fayette County,11,38.047430,-84.493511,30460,"Lexington-Fayette, KY",1,336,"Lexington-Fayette--Richmond--Frankfort, KY",N,N,2106,21077,21013,2019-2020,Z0516931
21570,NaN,NaN,FORT WORTH CHRISTIAN SCHOOL,NaN,6200 HOLIDAY LN,NORTH RICHLAND HILLS,TX,76180,48,48439,Tarrant County,21,32.857358,-97.220487,19100,"Dallas-Fort Worth-Arlington, TX",1,206,"Dallas-Fort Worth, TX-OK",N,N,4812,48091,48009,2019-2020,Z1326859


In [6]:
df_address_list_consolidated = df_address_list[[
    'STREET', 'CITY', 'STATE', 'ZIP', 'LAT', 'LON'
    ]].copy()
df_address_list_consolidated.columns = [
    column.title() for column in df_address_list_consolidated.columns]
df_address_list_consolidated

,Street,City,State,Zip,Lat,Lon
0,600 E Alabama Ave,Albertville,AL,35950,34.260200,-86.206200
1,402 E McCord Ave,Albertville,AL,35950,34.262200,-86.204900
2,901 W McKinney Ave,Albertville,AL,35950,34.273300,-86.220100
3,145 West End Drive,Albertville,AL,35950,34.252700,-86.221806
4,257 Country Club Rd,Albertville,AL,35951,34.289800,-86.193300
...,...,...,...,...,...,...
21567,221 VIOLA RD,MONSEY,NY,10952,41.135509,-74.078211
21568,6363 S TRENTON AVE,TULSA,OK,74136,36.069900,-95.965841
21569,194 N LIMESTONE,LEXINGTON,KY,40507,38.047430,-84.493511
21570,6200 HOLIDAY LN,NORTH RICHLAND HILLS,TX,76180,32.857358,-97.220487


In [7]:
df_address_list_consolidated.dtypes

Street     object
City       object
State      object
Zip        object
Lat       float64
Lon       float64
dtype: object

In [8]:
# Combining each street, city, state, and zip into an 'Address' entry:
df_address_list_consolidated['Address'] = \
(df_address_list_consolidated['Street'] + ', ' + 
df_address_list_consolidated['City'] + ', ' + 
df_address_list_consolidated['State'] + ' ' + 
df_address_list_consolidated['Zip'])
df_address_list_consolidated


,Street,City,State,Zip,Lat,Lon,Address
0,600 E Alabama Ave,Albertville,AL,35950,34.260200,-86.206200,"600 E Alabama Ave, Albertville, AL 35950"
1,402 E McCord Ave,Albertville,AL,35950,34.262200,-86.204900,"402 E McCord Ave, Albertville, AL 35950"
2,901 W McKinney Ave,Albertville,AL,35950,34.273300,-86.220100,"901 W McKinney Ave, Albertville, AL 35950"
3,145 West End Drive,Albertville,AL,35950,34.252700,-86.221806,"145 West End Drive, Albertville, AL 35950"
4,257 Country Club Rd,Albertville,AL,35951,34.289800,-86.193300,"257 Country Club Rd, Albertville, AL 35951"
...,...,...,...,...,...,...,...
21567,221 VIOLA RD,MONSEY,NY,10952,41.135509,-74.078211,"221 VIOLA RD, MONSEY, NY 10952"
21568,6363 S TRENTON AVE,TULSA,OK,74136,36.069900,-95.965841,"6363 S TRENTON AVE, TULSA, OK 74136"
21569,194 N LIMESTONE,LEXINGTON,KY,40507,38.047430,-84.493511,"194 N LIMESTONE, LEXINGTON, KY 40507"
21570,6200 HOLIDAY LN,NORTH RICHLAND HILLS,TX,76180,32.857358,-97.220487,"6200 HOLIDAY LN, NORTH RICHLAND HILLS, TX 76180"


In [9]:
df_va_addresses = df_address_list_consolidated.query("State == 'VA'").copy()
df_va_addresses

,Street,City,State,Zip,Lat,Lon,Address
92946,1000 East Rio Rd,Charlottesville,VA,22901,38.062500,-78.463800,"1000 East Rio Rd, Charlottesville, VA 22901"
92947,105 E Country Club Lane,Covington,VA,24426,37.788133,-79.909846,"105 E Country Club Lane, Covington, VA 24426"
92948,325 Pleasant Valley Rd,Harrisonburg,VA,22801,38.401100,-78.902100,"325 Pleasant Valley Rd, Harrisonburg, VA 22801"
92949,49 Hornet Road,Fishersville,VA,22939,38.116263,-78.989196,"49 Hornet Road, Fishersville, VA 22939"
92950,13400 Woodside Ln,Newport News,VA,23608,37.171400,-76.527000,"13400 Woodside Ln, Newport News, VA 23608"
...,...,...,...,...,...,...,...
21401,8918 TIDEWATER DR,NORFOLK,VA,23503,36.939590,-76.254131,"8918 TIDEWATER DR, NORFOLK, VA 23503"
21481,1071 WOODBERRY SQ PL,LYNCHBURG,VA,24502,37.388480,-79.242011,"1071 WOODBERRY SQ PL, LYNCHBURG, VA 24502"
21482,11579 CEDAR CHASE RD,HERNDON,VA,20170,38.997910,-77.344201,"11579 CEDAR CHASE RD, HERNDON, VA 20170"
21483,1030 BIBLE LN,VINTON,VA,24179,37.286660,-79.834681,"1030 BIBLE LN, VINTON, VA 24179"


In [10]:
# Creating a GeoDataFrame so that we can visualize each address on a map:
gdf_va_addresses = geopandas.GeoDataFrame(df_va_addresses, 
geometry = geopandas.points_from_xy(df_va_addresses['Lon'], 
df_va_addresses['Lat']))
# See https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html
gdf_va_addresses

,Street,City,State,Zip,Lat,Lon,Address,geometry
92946,1000 East Rio Rd,Charlottesville,VA,22901,38.062500,-78.463800,"1000 East Rio Rd, Charlottesville, VA 22901",POINT (-78.46380 38.06250)
92947,105 E Country Club Lane,Covington,VA,24426,37.788133,-79.909846,"105 E Country Club Lane, Covington, VA 24426",POINT (-79.90985 37.78813)
92948,325 Pleasant Valley Rd,Harrisonburg,VA,22801,38.401100,-78.902100,"325 Pleasant Valley Rd, Harrisonburg, VA 22801",POINT (-78.90210 38.40110)
92949,49 Hornet Road,Fishersville,VA,22939,38.116263,-78.989196,"49 Hornet Road, Fishersville, VA 22939",POINT (-78.98920 38.11626)
92950,13400 Woodside Ln,Newport News,VA,23608,37.171400,-76.527000,"13400 Woodside Ln, Newport News, VA 23608",POINT (-76.52700 37.17140)
...,...,...,...,...,...,...,...,...
21401,8918 TIDEWATER DR,NORFOLK,VA,23503,36.939590,-76.254131,"8918 TIDEWATER DR, NORFOLK, VA 23503",POINT (-76.25413 36.93959)
21481,1071 WOODBERRY SQ PL,LYNCHBURG,VA,24502,37.388480,-79.242011,"1071 WOODBERRY SQ PL, LYNCHBURG, VA 24502",POINT (-79.24201 37.38848)
21482,11579 CEDAR CHASE RD,HERNDON,VA,20170,38.997910,-77.344201,"11579 CEDAR CHASE RD, HERNDON, VA 20170",POINT (-77.34420 38.99791)
21483,1030 BIBLE LN,VINTON,VA,24179,37.286660,-79.834681,"1030 BIBLE LN, VINTON, VA 24179",POINT (-79.83468 37.28666)


In [11]:
m = folium.Map(location=[38, -79], zoom_start = 7, tiles='Stamen Terrain') 
# https://github.com/python-visualization/folium/blob/main/examples/Quickstart.ipynb

for i in range(len(gdf_va_addresses)):
    coords = gdf_va_addresses.iloc[i]['geometry']
    address = gdf_va_addresses.iloc[i]['Address']
    # print(coords)
    folium.CircleMarker(
        location = [coords.y, coords.x],
        radius = 5,
        tooltip= address,
        fill = True).add_to(m)
    # For the use of coords.y and coords.x, see:
    # https://shapely.readthedocs.io/en/stable/reference/shapely.Point.html

m

In [12]:
student_id_column = [i for i in range(1, 4001)]
male_first_name_list = [fake.first_name_male() for i in range(2000)]
# See https://faker.readthedocs.io/en/master/providers/faker.providers.person.html
female_first_name_list = [fake.first_name_female() for i in range(2000)]
first_name_column = male_first_name_list + female_first_name_list
gender_column = ['Male' for i in range(2000)] + ['Female' for i in range(2000)]
school_column = [rng.choice([
'Sycamore Academy', 'Dogwood Academy', 'Chestnut Academy', 'Hickory Academy'
]) for i in range(4000)]
# Creating a school acronym column by splitting each school name, 
# then combining the first letter of each word together:
acronym_column = [school.split(' ')[0][0] + school.split(' ')[1][0]
for school in school_column]
last_name_column = [fake.last_name() for i in range(4000)]
grade_column = [rng.choice([
'K', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'
]) for i in range(4000)]
race_column = [rng.choice(['African American', 'American Indian', 
'Asian', 'White'], p = [0.3, 0.05, 0.2, 0.45]) for i in range(4000)]
ethnicity_column = [rng.choice(['Hispanic', 'Non-Hispanic'], 
p = [0.3, 0.7]) for i in range(4000)]

In [13]:
df_curr_enrollment = pd.DataFrame({'Student_ID':student_id_column,
'First_Name':first_name_column, 'Last_Name':last_name_column, 
'Full_School_Name':school_column, 'School':acronym_column, 'Grade':grade_column, 
'Gender':gender_column, 'Race':race_column, 'Ethnicity':ethnicity_column})
df_curr_enrollment

,Student_ID,First_Name,Last_Name,Full_School_Name,School,Grade,Gender,Race,Ethnicity
0,1,James,Adams,Chestnut Academy,CA,9,Male,White,Non-Hispanic
1,2,Joseph,Moore,Sycamore Academy,SA,10,Male,African American,Hispanic
2,3,Kristopher,Williams,Hickory Academy,HA,6,Male,African American,Non-Hispanic
3,4,Noah,Rogers,Chestnut Academy,CA,6,Male,White,Hispanic
4,5,Daniel,Mcpherson,Chestnut Academy,CA,8,Male,White,Hispanic
...,...,...,...,...,...,...,...,...,...
3995,3996,Christina,Lindsey,Sycamore Academy,SA,9,Female,African American,Non-Hispanic
3996,3997,Lisa,Palmer,Sycamore Academy,SA,4,Female,African American,Non-Hispanic
3997,3998,Julia,Johnston,Hickory Academy,HA,2,Female,Asian,Non-Hispanic
3998,3999,Yvonne,Turner,Sycamore Academy,SA,7,Female,Asian,Non-Hispanic


I'll now merge in address info by:
1. Duplicating the list so that there are enough addresses for each student
1. Assigning a student_id value to each row of df_va_addresses
1. Merging the duplicated address list into the student list

In [14]:
df_va_addresses_for_merge = pd.concat([df_va_addresses, df_va_addresses])
df_va_addresses_for_merge['Student_ID'] = [i for i in range(len(df_va_addresses_for_merge))]
df_va_addresses_for_merge

,Street,City,State,Zip,Lat,Lon,Address,geometry,Student_ID
92946,1000 East Rio Rd,Charlottesville,VA,22901,38.062500,-78.463800,"1000 East Rio Rd, Charlottesville, VA 22901",POINT (-78.46380 38.06250),0
92947,105 E Country Club Lane,Covington,VA,24426,37.788133,-79.909846,"105 E Country Club Lane, Covington, VA 24426",POINT (-79.90985 37.78813),1
92948,325 Pleasant Valley Rd,Harrisonburg,VA,22801,38.401100,-78.902100,"325 Pleasant Valley Rd, Harrisonburg, VA 22801",POINT (-78.90210 38.40110),2
92949,49 Hornet Road,Fishersville,VA,22939,38.116263,-78.989196,"49 Hornet Road, Fishersville, VA 22939",POINT (-78.98920 38.11626),3
92950,13400 Woodside Ln,Newport News,VA,23608,37.171400,-76.527000,"13400 Woodside Ln, Newport News, VA 23608",POINT (-76.52700 37.17140),4
...,...,...,...,...,...,...,...,...,...
21401,8918 TIDEWATER DR,NORFOLK,VA,23503,36.939590,-76.254131,"8918 TIDEWATER DR, NORFOLK, VA 23503",POINT (-76.25413 36.93959),5511
21481,1071 WOODBERRY SQ PL,LYNCHBURG,VA,24502,37.388480,-79.242011,"1071 WOODBERRY SQ PL, LYNCHBURG, VA 24502",POINT (-79.24201 37.38848),5512
21482,11579 CEDAR CHASE RD,HERNDON,VA,20170,38.997910,-77.344201,"11579 CEDAR CHASE RD, HERNDON, VA 20170",POINT (-77.34420 38.99791),5513
21483,1030 BIBLE LN,VINTON,VA,24179,37.286660,-79.834681,"1030 BIBLE LN, VINTON, VA 24179",POINT (-79.83468 37.28666),5514


In [15]:
df_curr_enrollment = df_curr_enrollment.merge(df_va_addresses_for_merge, on = 'Student_ID', how = 'left')
df_curr_enrollment.sort_values(['School', 'Grade', 'Last_Name', 'First_Name'], inplace = True)
df_curr_enrollment['Students'] = 1 # Useful for pivot tables
# I encountered difficulties trying to upload the 'geometry' column to my
# SQL database, so I'll remove it here.
df_curr_enrollment.drop('geometry', axis = 1, inplace = True)
df_curr_enrollment['Grade_for_Sorting'] = df_curr_enrollment['Grade'].copy()
df_curr_enrollment['Grade_for_Sorting'].replace({'K':'0'}, inplace = True)
df_curr_enrollment['Grade_for_Sorting'] = df_curr_enrollment['Grade_for_Sorting'].astype('int')

df_curr_enrollment

,Student_ID,First_Name,Last_Name,Full_School_Name,School,Grade,Gender,Race,Ethnicity,Street,City,State,Zip,Lat,Lon,Address,Students,Grade_for_Sorting
1137,1138,Daniel,Atkins,Chestnut Academy,CA,1,Male,Asian,Non-Hispanic,701 Potomac Station Dr NE,Leesburg,VA,20176,39.094800,-77.522700,"701 Potomac Station Dr NE, Leesburg, VA 20176",1,1
3488,3489,Michelle,Banks,Chestnut Academy,CA,1,Female,White,Non-Hispanic,13340 Leland Rd.,Centreville,VA,20120,38.845700,-77.408300,"13340 Leland Rd., Centreville, VA 20120",1,1
2402,2403,Brenda,Bell,Chestnut Academy,CA,1,Female,Asian,Hispanic,3846 KING ST,ALEXANDRIA,VA,22302,38.831460,-77.094591,"3846 KING ST, ALEXANDRIA, VA 22302",1,1
1631,1632,Michael,Bell,Chestnut Academy,CA,1,Male,African American,Non-Hispanic,1301 Whitehead Rd,Richmond,VA,23225,37.489169,-77.508759,"1301 Whitehead Rd, Richmond, VA 23225",1,1
1515,1516,James,Bishop,Chestnut Academy,CA,1,Male,African American,Non-Hispanic,3051 Old Bridge Road,Woodbridge,VA,22192,38.682400,-77.305100,"3051 Old Bridge Road, Woodbridge, VA 22192",1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3720,3721,Samantha,Weiss,Sycamore Academy,SA,K,Female,White,Non-Hispanic,9200 Lydell Dr,Richmond,VA,23228,37.642000,-77.481900,"9200 Lydell Dr, Richmond, VA 23228",1,0
2271,2272,Kelly,Wilkerson,Sycamore Academy,SA,K,Female,Asian,Hispanic,279 TANDEM LN,CHARLOTTESVILLE,VA,22902,37.995788,-78.491879,"279 TANDEM LN, CHARLOTTESVILLE, VA 22902",1,0
168,169,Justin,Wilson,Sycamore Academy,SA,K,Male,White,Hispanic,575 Union Hill Rd,Amherst,VA,24521,37.569400,-79.036200,"575 Union Hill Rd, Amherst, VA 24521",1,0
2299,2300,Elizabeth,Woods,Sycamore Academy,SA,K,Female,Asian,Non-Hispanic,227 S CHERRY ST,RICHMOND,VA,23220,37.542010,-77.452541,"227 S CHERRY ST, RICHMOND, VA 23220",1,0


In [16]:
df_curr_enrollment.to_csv('curr_enrollment.csv', index = False)

# Exporting this table to an ElephantSQL database instance:

## Connecting to the database:

In [17]:
with open ("../key_paths/path_to_keys_folder.txt") as file:
    key_path = file.read()

with open(key_path+"/elephantsql_dashschooldemodb_url.txt") as file:
    db_url = file.read()
    # This code reads in my database's URL, which is listed on the home page for my database within elephantsql.com. As shown below, SQLAlchemy can use this URL to connect to the database. 

elephantsql_db_url_for_sqlalchemy = db_url.replace('postgres://', 'postgresql://')
# This change, which is required for SQLAlchemy to work correctly, is based on the code suggested at:
# # https://help.heroku.com/ZKNTJQSK/why-is-sqlalchemy-1-4-x-not-connecting-to-heroku-postgres

elephantsql_engine = sqlalchemy.create_engine(elephantsql_db_url_for_sqlalchemy)

Uploading df_curr_enrollment_to_SQL:

In [18]:
df_curr_enrollment.to_sql("curr_enrollment",if_exists = 'replace', con = elephantsql_engine, index = False)

1000

Ensuring that the upload worked as expected:

In [19]:
pd.read_sql("curr_enrollment", con = elephantsql_engine)

,Student_ID,First_Name,Last_Name,Full_School_Name,School,Grade,Gender,Race,Ethnicity,Street,City,State,Zip,Lat,Lon,Address,Students,Grade_for_Sorting
0,1138,Daniel,Atkins,Chestnut Academy,CA,1,Male,Asian,Non-Hispanic,701 Potomac Station Dr NE,Leesburg,VA,20176,39.094800,-77.522700,"701 Potomac Station Dr NE, Leesburg, VA 20176",1,1
1,3489,Michelle,Banks,Chestnut Academy,CA,1,Female,White,Non-Hispanic,13340 Leland Rd.,Centreville,VA,20120,38.845700,-77.408300,"13340 Leland Rd., Centreville, VA 20120",1,1
2,2403,Brenda,Bell,Chestnut Academy,CA,1,Female,Asian,Hispanic,3846 KING ST,ALEXANDRIA,VA,22302,38.831460,-77.094591,"3846 KING ST, ALEXANDRIA, VA 22302",1,1
3,1632,Michael,Bell,Chestnut Academy,CA,1,Male,African American,Non-Hispanic,1301 Whitehead Rd,Richmond,VA,23225,37.489169,-77.508759,"1301 Whitehead Rd, Richmond, VA 23225",1,1
4,1516,James,Bishop,Chestnut Academy,CA,1,Male,African American,Non-Hispanic,3051 Old Bridge Road,Woodbridge,VA,22192,38.682400,-77.305100,"3051 Old Bridge Road, Woodbridge, VA 22192",1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3721,Samantha,Weiss,Sycamore Academy,SA,K,Female,White,Non-Hispanic,9200 Lydell Dr,Richmond,VA,23228,37.642000,-77.481900,"9200 Lydell Dr, Richmond, VA 23228",1,0
3996,2272,Kelly,Wilkerson,Sycamore Academy,SA,K,Female,Asian,Hispanic,279 TANDEM LN,CHARLOTTESVILLE,VA,22902,37.995788,-78.491879,"279 TANDEM LN, CHARLOTTESVILLE, VA 22902",1,0
3997,169,Justin,Wilson,Sycamore Academy,SA,K,Male,White,Hispanic,575 Union Hill Rd,Amherst,VA,24521,37.569400,-79.036200,"575 Union Hill Rd, Amherst, VA 24521",1,0
3998,2300,Elizabeth,Woods,Sycamore Academy,SA,K,Female,Asian,Non-Hispanic,227 S CHERRY ST,RICHMOND,VA,23220,37.542010,-77.452541,"227 S CHERRY ST, RICHMOND, VA 23220",1,0
